In [ ]:
import librosa
import matplotlib.pyplot as plt
import IPython.display as dis
import numpy as np
import os, sys
from tqdm import tqdm
import pandas as pd
import librosa.display

from google.colab import drive
drive.mount('/content/gdrive')

!unzip /content/gdrive/MyDrive/final/melspectrogram_data.zip -d /content/melspectrogram_data

rawdata = pd.read_csv('/content/gdrive/MyDrive/final/Raw_data.csv')

lst = os.listdir('/content/melspectrogram_data')
data_x = []
data_y = []
for i in tqdm(range(len(rawdata))):
   data_x.append(np.load('/content/melspectrogram_data/' + str(rawdata['song_id'].iloc[i])+'.npy'))
   data_y.append(rawdata['senti_tag'].iloc[i])

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data_y = label_encoder.fit_transform(data_y)

wav_to_img_data = []
target = []

# 128*128 사이즈로 멜스펙토그램 이미지를 잘라서 담아두고
# 타겟값을 같이 추가한 넘파이배열을 만든다
for idx in tqdm(range(len(data_x))):
    for i in range(15):
        wav_to_img_data.append(data_x[idx][:, 128* i : 128 * (i+1)].reshape(128,128,-1))
        target.append(data_y[idx])
wav_to_img_data = np.array(wav_to_img_data)
target = np.array(target)

%cd /content/gdrive/MyDrive/final
import vgg
import tensorflow as tf

model = vgg.VGG_model()

model.summary()

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',  metrics=['acc'])

history = model.fit(wav_to_img_data, target, epochs=20, batch_size=32, validation_split=0.2)

model.save('/content/gdrive/MyDrive/final/VGG_128by128.h5')

music = []
# for idx in tqdm(range(len(data_x))):
idx = 645
for i in range(len(data_x[idx][0])):
    if i+128 == len(data_x[idx][0]):
        break
    music.append(data_x[idx][:, i : 128 + i].reshape(128,128,-1))

result = model.predict(np.array(music))

res = np.argmax(result, axis=1)

np.unique(res, return_counts=True)